In [56]:
#creating KxP hard assignment matrix for the anatomical thalamus segmentations 

import Functional_Fusion.atlas_map as am
import HierarchBayesParcel.arrangements as ar
import nibabel as  nb
import numpy as np 
import torch as pt 

wk_dir = '/Users/incehusain/fs_projects'

iglesias_atlas_nii = f"{wk_dir}/dseg_segmentations_Language7T/sub_19_thalamus_dseg.nii.gz"

lut_file = f"{wk_dir}/compressionLookupTable.txt"

iglesias_head = '# Labels for thalamus parcellation using histological atlas (Iglesias et al.)'

lut_label_to_name = {}
with open(lut_file, 'r') as f:
    for line in f:
        if line.strip() and not line.startswith('#'):
            parts = line.strip().split()
            label = int(parts[0])
            name = parts[1]
            lut_label_to_name[label] = name
thalamus_labels = sorted(list(lut_label_to_name.keys()))

K = len(thalamus_labels)    

img = nb.load(iglesias_atlas_nii)   
data = img.get_fdata().astype(np.int32)

thalamus_voxel = data > 0 

voxel_labels = data[thalamus_voxel]

P=len(voxel_labels)

#we are no longer reading it into MNISymThalamus1 space - going to make new ROI mask using anatomical images
#atlas, _ = am.get_atlas('MNISymThalamus1')

#U = atlas.read_data(iglesias_atlas_nii)

#P=U.shape[0]

label_to_index = {label: idx for idx, label in enumerate(thalamus_labels)}

prob_map_numpy = np.zeros((K, P))

for p, label in enumerate(voxel_labels):
    if label in label_to_index:
        k = label_to_index[label]
        prob_map_numpy[k, p] = 1.0

U_iglesias = np.array(prob_map_numpy)

U_iglesias_map = np.round(U_iglesias)

#ar_model = ar.build_arrangement_model(U_iglesias_map, prior_type='prob', atlas=atlas)

#print("TEST")


KeyboardInterrupt: 

In [9]:
import ants 
import os

wk_dir = '/Users/incehusain/fs_projects'

sub = 'sub-02'

output_prefix = f"{wk_dir}/{sub}_to_MNI_"

template = f"{wk_dir}/tpl-MNI152NLin2009cSym_res-1_T1w.nii"
T1_nii = f"{wk_dir}/freesurfer_Language7T/{sub}/mri/T1.mgz"   #or norm.mgz
seg_nii = f"{wk_dir}/dseg_segmentations_Language7T/{sub}_thalamus_dseg.nii.gz"

output_path = f"{wk_dir}"

template_ants = ants.image_read(template)
T1 = ants.image_read(T1_nii)
seg = ants.image_read(seg_nii)

print('Registering...')

registration = ants.registration(fixed=template_ants, moving=T1, type_of_transform='SyN', outprefix=output_prefix)

print('Registration done!')

warped_seg = ants.apply_transforms(fixed=template_ants, moving=seg, transformlist=registration['fwdtransforms'], interpolator='linear')

ants.image_write(warped_seg, f"{wk_dir}/{sub}_seg_warped.nii.gz")



Registering...
Registration done!
